In [ ]:
# ================================
# 1) Imports and Setup
# ================================
!pip install -q transformers accelerate faiss-cpu

import os
import json
import numpy as np
import pandas as pd
import torch
import faiss
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
from google.colab import drive
drive.mount('/content/drive')

# ================================
# 2) Configuration
# ================================
BASE_DIR = "/content/drive/MyDrive/CIDR Projects/PoliticalAnalysis/חוק משק החשמל/RAG"
INDEX_PATH = os.path.join(BASE_DIR, "all_protocols_parsed_by_speaker.faiss")
META_PATH  = os.path.join(BASE_DIR, "all_protocols_parsed_by_speaker_metadata.parquet")
DIFF_PATH  = os.path.join(BASE_DIR, "cotent_offer_and_final_format_comparison_annotated - cotent_offer_and_final_format_comparison.csv")

OUTPUT_JSON = os.path.join(BASE_DIR, "electricity_law_diff_debates_report.json")

device = "cuda" if torch.cuda.is_available() else "cpu"


# ================================
# 3) Load Data and Models
# ================================
index = faiss.read_index(INDEX_PATH)
df_meta = pd.read_parquet(META_PATH)
df_diff = pd.read_csv(DIFF_PATH)

print(f"FAISS index loaded: {index.ntotal} vectors")
print(f"Metadata loaded: {len(df_meta)} rows")
print(f"Differences table loaded: {len(df_diff)} rows")


# ================================
# 4) Embedding Model (Qwen Encoder)
# ================================
ENCODER_MODEL = "Qwen/Qwen3-Embedding-0.6B"

encoder_tokenizer = AutoTokenizer.from_pretrained(ENCODER_MODEL)
encoder_model = AutoModel.from_pretrained(
    ENCODER_MODEL,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
).to(device)
encoder_model.eval()


def encode_texts(texts, max_length=512, batch_size=16):
    """Returns numpy embeddings for a list of texts."""
    vectors = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        tokens = encoder_tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=max_length).to(device)

        with torch.no_grad():
            output = encoder_model(**tokens).last_hidden_state
            mask = tokens["attention_mask"].unsqueeze(-1)
            pooled = (output * mask).sum(dim=1) / mask.sum(dim=1).clamp(min=1e-9)

        vectors.append(pooled.cpu().numpy())

    return np.vstack(vectors)


def embed_single(text):
    """Convenience wrapper for a single text embedding."""
    return encode_texts([text])


# ================================
# 5) FAISS Search Utilities
# ================================
def search(query, k, index_obj, metadata_df):
    """Returns top-k matches with scores and associated metadata."""
    q_vec = embed_single(query)
    scores, idxs = index_obj.search(q_vec, k)
    hits = metadata_df.iloc[idxs[0]].copy()
    hits["score"] = scores[0]
    return hits.sort_values("score", ascending=False)


def get_context(hit, df, window=5):
    """Returns contextual transcript window around a retrieved line."""
    condition = (
        (df["protocol_number"] == hit["protocol_number"]) &
        (df["utterance_idx"].between(hit["utterance_idx"] - window, hit["utterance_idx"] + window))
    )
    ctx = df.loc[condition].sort_values(["protocol_number", "utterance_idx"])
    lines = []

    for _, r in ctx.iterrows():
        sp = str(r.get("speaker", "")).strip()
        txt = str(r.get("text", "")).strip()
        lines.append(f"{sp}: {txt}" if sp else txt)

    return "\n".join(lines)


# ================================
# 6) Retrieve up to 5 debates per difference
# ================================
import pandas as pd
def retrieve_debates(
    row,
    diff_index,
    index,
    metadata_df,
    max_search_hits=200,   # כמה מועמדים למשוך מ-FAISS
    top_k=20,              # כמה דיונים לשמור בפועל
    min_score=0.85,
    window=5
):
    """
    Retrieve top-K debates per difference, ranked by normalized similarity.
    """

    # ----------------------------
    # 1. Validate difference text
    # ----------------------------
    diff_raw = row.get(f"הבדל{diff_index}", None)

    if diff_raw is None or pd.isna(diff_raw):
        return [], {"hits_total": 0, "above_threshold": 0, "top_raw": None}

    diff_text = str(diff_raw).strip()
    if diff_text.lower() in ["", "nan", "none"]:
        return [], {"hits_total": 0, "above_threshold": 0, "top_raw": None}

    section_title = str(row.get("שם סעיף", "")).strip()

    # ----------------------------
    # 2. FAISS search
    # ----------------------------
    query = (
        f"דיון בעניין סעיף {section_title}. "
        f"הפער בין נוסח ההצעה לחוק לבין החוק הסופי הוא: {diff_text}"
    )

    hits = search(query, max_search_hits, index, metadata_df)

    if hits.empty:
        return [], {"hits_total": 0, "above_threshold": 0, "top_raw": None}

    # ----------------------------
    # 3. Normalize scores
    # ----------------------------
    top_raw = float(hits["score"].max())
    hits["score_norm"] = hits["score"] / top_raw if top_raw > 0 else 0.0

    valid_hits = hits[hits["score_norm"] >= min_score]

    # ----------------------------
    # 4. Take TOP-K highest matches
    # ----------------------------
    selected_hits = (
        valid_hits
        .sort_values("score_norm", ascending=False)
        .head(top_k)
    )

    debates = []

    for _, h in selected_hits.iterrows():
        debate_uid = f"{int(h['protocol_number'])}_{int(h['utterance_idx'])}"

        debates.append({
            "debate_uid": debate_uid,
            "protocol_number": int(h["protocol_number"]),
            "initiator": str(h.get("speaker", "")).strip(),
            "retrieval_score": float(h["score_norm"]),
            "transcript": get_context(h, metadata_df, window),
        })

    stats = {
        "hits_total": len(hits),
        "above_threshold": len(valid_hits),
        "returned": len(debates),
        "top_raw": top_raw,
        "min_score": min_score,
    }

    return debates, stats



# # ================================
# # 7) Debate LLM Analysis (quote + stance extraction)
# # ================================
# LLM_MODEL = "Qwen/Qwen2-7B-Instruct"

# llm_tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL)
# llm_model = AutoModelForCausalLM.from_pretrained(
#     LLM_MODEL,
#     torch_dtype=torch.float16 if device=="cuda" else torch.float32,
#     device_map="auto")


# def analyze_debate(diff_text, transcript, max_new_tokens=512):
#     """LLM detects relevance, summarizes and extracts supporters/opposers with quotes."""

#     speakers = sorted({line.split(":")[0].strip() for line in transcript.splitlines() if ":" in line})
#     allowed = ", ".join(speakers) if speakers else "אין דוברים"

#     prompt = f"""
# אתה מנתח פרוטוקול של ועדת כנסת.

# השינוי הנבדק:
# \"\"\"{diff_text}\"\"\"

# תמלול:
# \"\"\"{transcript}\"\"\

# דוברים מותרים:
# {allowed}

# משימה:
# - קבע האם הדיון קשור לשינוי (גבוה / בינוני / לא קשור)
# - אם יש קשר: סכם בקצרה
# - זהה תומכים ומתנגדים והשב עם ציטוט מדויק שלהם מהטקסט

# החזר אך ורק JSON:

# {{
#  "relevance": "",
#  "summary": "",
#  "supporters": [{{"speaker": "", "quote": ""}}],
#  "opponents":  [{{"speaker": "", "quote": ""}}]
# }}
# """
#     inputs = llm_tokenizer(prompt, return_tensors="pt").to(device)
#     output = llm_model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
#     text = llm_tokenizer.decode(output[0], skip_special_tokens=True)

#     try:
#         json_part = text[text.index("{"): text.rindex("}")+1]
#         out = json.loads(json_part)
#     except:
#         return {"relevance": "parse_error", "summary": "", "supporters": [], "opponents": []}

#     # filter only allowed speakers
#     out["supporters"] = [p for p in out.get("supporters", []) if p.get("speaker") in speakers]
#     out["opponents"] = [p for p in out.get("opponents", []) if p.get("speaker") in speakers]

#     return out


# ================================
# 8) Build structured JSON per section
# ================================
def build_section_json(row, index, metadata_df):
    """
    Build clean JSON structure for a single legal section.
    Only includes differences that have at least one valid debate.
    """

    section = {
        "section_name": str(row.get("שם סעיף", "")).strip(),
        "proposal_text": str(row.get("הצעת חוק", "")).strip(),
        "final_text": str(row.get("חוק סופי", "")).strip(),
        "gaps": []
    }

    # --------------------------------
    # Iterate over possible differences
    # --------------------------------
    for diff_index in range(1, 5):

        debates, stats = retrieve_debates(
            row=row,
            diff_index=diff_index,
            index=index,
            metadata_df=metadata_df
        )

        # --------------------------------
        # Invariant: only gaps with debates
        # --------------------------------
        if len(debates) == 0:
            continue

        diff_text = str(row.get(f"הבדל{diff_index}")).strip()

        gap_obj = {
            "gap_index": diff_index,
            "gap_text": diff_text,
            "retrieval_stats": stats,
            "debates": []
        }

        for j, d in enumerate(debates, start=1):
            gap_obj["debates"].append({
                "debate_index": j,
                "debate_uid": d["debate_uid"],
                "protocol_number": d["protocol_number"],
                "initiator": d["initiator"],
                "retrieval_score": d["retrieval_score"],
                "transcript": d["transcript"]
            })

        section["gaps"].append(gap_obj)

    return section





# ================================
# 9) Run over all rows and export JSON
# ================================
results = []

for i, row in tqdm(df_diff.iterrows(), total=len(df_diff)):
    print(f"Processing section {i+1}/{len(df_diff)}:", row.get("שם סעיף", ""))
    #esults.append(build_section_json(row))
    section_json = build_section_json(row, index, df_meta)
    # Keep only sections with at least one gap
    if len(section_json["gaps"]) > 0:
        results.append(section_json)


with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print("Completed.")
print("Saved to:", OUTPUT_JSON)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
FAISS index loaded: 2604 vectors
Metadata loaded: 2604 rows
Differences table loaded: 63 rows


  0%|          | 0/63 [00:00<?, ?it/s]

Processing section 1/63: מטרת החוק


  2%|▏         | 1/63 [00:00<00:51,  1.21it/s]

Processing section 2/63: הגדרות


...

Processing section 57/63: שמירת התחייבויות וזכויות
Processing section 58/63: העברת זכויות ונכסים
Processing section 59/63: ביצוע כללים ותקנות


 97%|█████████▋| 61/63 [00:08<00:00, 10.30it/s]

Processing section 60/63: תיקון חוק החברות הממשלתיות
Processing section 61/63: תיקון חוק העבירות המינהליות ­ מס' 3
Processing section 62/63: תיקון חוק שירות המדינה (מינויים) ­ מס' 9


100%|██████████| 63/63 [00:08<00:00,  7.51it/s]


Processing section 63/63: תחילה
Completed.
Saved to: /content/drive/MyDrive/CIDR Projects/PoliticalAnalysis/חוק משק החשמל/RAG/electricity_law_diff_debates_report.json


In [ ]:
import json
import pandas as pd

BASE_DIR = "/content/drive/MyDrive/CIDR Projects/PoliticalAnalysis/חוק משק החשמל/RAG"
JSON_PATH = f"{BASE_DIR}/electricity_law_diff_debates_report.json"
OUTPUT_XLSX = f"{BASE_DIR}/electricity_law_diff_tagging_template.xlsx"

with open(JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"Loaded {len(data)} sections")


Loaded 62 sections


In [ ]:
rows = []

for section in data:
    section_name = section.get("section_name", "").strip()

    for gap in section.get("gaps", []):
        gap_index = gap.get("gap_index")

        row_label = f"{section_name} | הבדל {gap_index}"

        row = {
            "section_name": section_name,
            "gap_index": gap_index,
            "row_label": row_label
        }

        # Create empty columns for up to 20 debates
        for i in range(1, 21):
            row[f"דיון_{i}"] = ""

        rows.append(row)

print(f"Created {len(rows)} gap rows")


Created 87 gap rows


In [ ]:
df = pd.DataFrame(rows)

# סדר עמודות יפה
ordered_columns = (
    ["section_name", "gap_index", "row_label"] +
    [f"דיון_{i}" for i in range(1, 21)]
)

df = df[ordered_columns]

df.to_excel(OUTPUT_XLSX, index=False)

print("Saved tagging template to:")
print(OUTPUT_XLSX)


Saved tagging template to:
/content/drive/MyDrive/CIDR Projects/PoliticalAnalysis/חוק משק החשמל/RAG/electricity_law_diff_tagging_template.xlsx


## Validation - (not ready to use cells, just some scatches i made with Noa)

In [ ]:
def find_sections_with_keyword(results, keyword):
    matches = []
    for i, sec in enumerate(results):
        name = sec.get("section_name", "")
        if keyword in name:
            matches.append((i, name))
    return matches


sections_with_horot = find_sections_with_keyword(results, "הוראות")

for idx, name in sections_with_horot:
    print(f"{idx:02d} | {name}")


In [ ]:
TARGET_SEC_IDX = 54


In [ ]:
def extract_retrieved_protocols(section):
    found = {}
    for gap in section["gaps"]:
        for deb in gap["debates"]:
            p = deb["protocol_number"]
            found.setdefault(p, []).append(deb)
    return found


sec = results[TARGET_SEC_IDX]
retrieved_protocols = extract_retrieved_protocols(sec)

print("Retrieved protocols:")
for p, debates in sorted(retrieved_protocols.items()):
    print(f"Protocol {p}: {len(debates)} debates")


In [ ]:
expert_protocols = {
    1: [2, 9, 10, 16, 30, 35],
    2: [4, 16, 18],
    3: [6, 9, 10, 19, 21, 27, 28, 33],
    4: [18],
    5: [16],
    6: [35, 45],
    7: [61, 62, 63, 65, 67, 68, 69, 70, 71],
    8: [5, 6, 8, 13],
}


def compare_with_expert(retrieved_protocols, expert_protocols):
    rows = []

    for proto in sorted(expert_protocols.keys()):
        rows.append({
            "protocol": proto,
            "expert_has_discussion": True,
            "retrieved": proto in retrieved_protocols,
            "n_retrieved_debates": len(retrieved_protocols.get(proto, [])),
        })

    # false positives (retrieved but expert didn't mention)
    for proto in retrieved_protocols:
        if proto not in expert_protocols:
            rows.append({
                "protocol": proto,
                "expert_has_discussion": False,
                "retrieved": True,
                "n_retrieved_debates": len(retrieved_protocols.get(proto, [])),
            })

    return pd.DataFrame(rows).sort_values("protocol")


comparison_df = compare_with_expert(retrieved_protocols, expert_protocols)
comparison_df


In [ ]:
tp = ((comparison_df.expert_has_discussion) & (comparison_df.retrieved)).sum()
fn = ((comparison_df.expert_has_discussion) & (~comparison_df.retrieved)).sum()
fp = ((~comparison_df.expert_has_discussion) & (comparison_df.retrieved)).sum()

precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0

print(f"TP (found & expected): {tp}")
print(f"FN (missed expert):   {fn}")
print(f"FP (extra protocols): {fp}")
print(f"Precision: {precision:.2f}")
print(f"Recall:    {recall:.2f}")


In [ ]:
import json
import pandas as pd
import os

# If you already have OUTPUT_JSON from the previous code, reuse it
OUTPUT_JSON = os.path.join(
    BASE_DIR,
    "electricity_law_diff_debates_report.json"
)

with open(OUTPUT_JSON, "r", encoding="utf-8") as f:
    results = json.load(f)

print(f"Loaded {len(results)} sections from JSON")


In [ ]:
def build_section_df(results):
    """
    Build a DataFrame with one row per legal section.
    """
    rows = []
    for sec_idx, sec in enumerate(results):
        gaps = sec.get("פערים", []) or []
        row = {
            "section_index": sec_idx,
            "section_name": sec.get("שם סעיף", ""),
            "proposal_text": sec.get("הסעיף לפי נוסח ההצעה", ""),
            "final_text": sec.get("הסעיף בנוסח החוק בפועל", ""),
            "n_gaps": len(gaps),
        }
        rows.append(row)
    return pd.DataFrame(rows)

section_df = build_section_df(results)
section_df.head()


In [ ]:
def build_gap_df(results):
    """
    Build a DataFrame with one row per gap (difference) in each section.
    """
    rows = []

    for sec_idx, sec in enumerate(results):
        section_name = sec.get("שם סעיף", "")
        gaps = sec.get("פערים", []) or []

        for gap in gaps:
            stats = gap.get("סטטיסטיקה", {}) or {}
            row = {
                "section_index": sec_idx,
                "section_name": section_name,
                "gap_index": gap.get("מספר פער", None),
                "gap_text": gap.get("תיאור הפער", ""),
                "hits_total": stats.get("hits_total", 0),
                "hits_above_threshold": stats.get("above_threshold", 0),
                "top_raw_score": stats.get("top_raw", None),
                "min_score": stats.get("min_score", None),
                "n_debates": len(gap.get("דיונים", []) or []),
            }
            rows.append(row)

    return pd.DataFrame(rows)

gap_df = build_gap_df(results)
gap_df.head()


In [ ]:
def build_debate_df(results):
    """
    Build a DataFrame with one row per debate for each gap and section.
    """
    rows = []

    for sec_idx, sec in enumerate(results):
        section_name = sec.get("שם סעיף", "")
        gaps = sec.get("פערים", []) or []

        for gap in gaps:
            gap_index = gap.get("מספר פער", None)
            gap_text = gap.get("תיאור הפער", "")
            debates = gap.get("דיונים", []) or []

            for deb in debates:
                row = {
                    "section_index": sec_idx,
                    "section_name": section_name,
                    "gap_index": gap_index,
                    "gap_text": gap_text,
                    "debate_index": deb.get("מספר דיון", None),
                    "protocol_number": deb.get("מספר פרוטוקול", None),
                    "initiator": deb.get("מי יזם את השיח", ""),
                    "retrieval_score": deb.get("ציון", None),
                    "relevance": deb.get("relevance", ""),
                    "summary": deb.get("summary", ""),
                    "n_supporters": len(deb.get("supporters", []) or []),
                    "n_opponents": len(deb.get("opponents", []) or []),
                }
                rows.append(row)

    return pd.DataFrame(rows)

debate_df = build_debate_df(results)
debate_df.head()


In [ ]:
def build_stance_df(results):
    """
    Build a DataFrame with one row per (speaker, stance) inside each debate.

    Columns include:
      - section_name, gap_text, debate_index, protocol_number
      - stance ("support" / "oppose")
      - speaker, quote
      - relevance (of the debate), retrieval_score
    """
    rows = []

    for sec_idx, sec in enumerate(results):
        section_name = sec.get("שם סעיף", "")
        gaps = sec.get("פערים", []) or []

        for gap in gaps:
            gap_index = gap.get("מספר פער", None)
            gap_text = gap.get("תיאור הפער", "")
            debates = gap.get("דיונים", []) or []

            for deb in debates:
                debate_index = deb.get("מספר דיון", None)
                protocol_number = deb.get("מספר פרוטוקול", None)
                initiator = deb.get("מי יזם את השיח", "")
                retrieval_score = deb.get("ציון", None)
                relevance = deb.get("relevance", "")
                summary = deb.get("summary", "")

                # Supporters
                for s in deb.get("supporters", []) or []:
                    rows.append({
                        "section_index": sec_idx,
                        "section_name": section_name,
                        "gap_index": gap_index,
                        "gap_text": gap_text,
                        "debate_index": debate_index,
                        "protocol_number": protocol_number,
                        "initiator": initiator,
                        "stance": "support",
                        "speaker": s.get("speaker", ""),
                        "quote": s.get("quote", ""),
                        "relevance": relevance,
                        "retrieval_score": retrieval_score,
                        "debate_summary": summary,
                    })

                # Opponents
                for o in deb.get("opponents", []) or []:
                    rows.append({
                        "section_index": sec_idx,
                        "section_name": section_name,
                        "gap_index": gap_index,
                        "gap_text": gap_text,
                        "debate_index": debate_index,
                        "protocol_number": protocol_number,
                        "initiator": initiator,
                        "stance": "oppose",
                        "speaker": o.get("speaker", ""),
                        "quote": o.get("quote", ""),
                        "relevance": relevance,
                        "retrieval_score": retrieval_score,
                        "debate_summary": summary,
                    })

    return pd.DataFrame(rows)

stance_df = build_stance_df(results)
stance_df.head()


In [ ]:
def list_sections(results):
    """
    Prints all section names with index numbers for selection.
    """
    print("Available sections:")
    print("-" * 60)
    for idx, sec in enumerate(results):
        name = sec.get("שם סעיף", "").strip()
        print(f"{idx:02d} | {name}")
    print("-" * 60)

list_sections(results)

def get_section_by_index(results, index):
    """
    Returns a single section JSON object by numeric index.
    """
    if index < 0 or index >= len(results):
        raise ValueError(f"Section index {index} is out of range (0-{len(results)-1}).")
    return results[index]


def pretty_print_section(section):
    """
    Displays a human-readable preview summary of a section.
    """
    print("=" * 80)
    print("SECTION SELECTED")
    print("=" * 80)

    print(f"Name: {section.get('שם סעיף', '').strip()}")
    print(f"Gaps found: {len(section.get('פערים', []))}")
    print("-" * 80)

    print("\nProposal Version:")
    print(section.get("הסעיף לפי נוסח ההצעה", "").strip())

    print("\nFinal Approved Version:")
    print(section.get("הסעיף בנוסח החוק בפועל", "").strip())

    print("=" * 80)


In [ ]:
# Step 1: show list
list_sections(results)

# Step 2: choose one manually (example: section #3)
chosen_index = 3

# Step 3: fetch object
selected_section = get_section_by_index(results, chosen_index)

# Step 4: display summary
pretty_print_section(selected_section)


In [ ]:
# Run this cell in Colab to choose interactively
list_sections(results)
chosen_index = int(input("\nEnter section index: "))
selected_section = get_section_by_index(results, chosen_index)
pretty_print_section(selected_section)


In [ ]:
debate_df.to_excel(os.path.join(BASE_DIR, "debate_level_table.xlsx"), index=False)
